# Visualizations of main optimization results

This notebook has been created to visualize the comparisons between the optimizations of the differently created models in the in silico study of the paper.

We compare here all the models run with 3 labels, but different original sets of parameters. Mainly, we reproduce Figure 4b and Figure 5b, 5c of the paper. But it can also be used to generally explore the dataset.

In [ ]:
# Importing al necessary functions
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import os
import numpy as np
import copy
import matplotlib
import matplotlib.ticker as ticker

from analyze_results import CumulativeResults
from scipy import stats

import seaborn as sns

In [ ]:
# We define some variables that will reoccur more often.
model_name = 'lipidomics_2023_08_30'
res_dict = '../Results_h5'

We start by loading all results of all different models into a singular object, that can then visualize comparisons etc

In [ ]:
# This function will load each individual hdf5 file. One file represents an optimization of a specific model.
cum_res = CumulativeResults(
    model_name, res_dict, n_res=1000
)

In [ ]:
# In addition to the hdf5 file we will be in need of the PEtab problem and the pyPESTO problem accordingly
cum_res.load_petab_problems("../Petab_models_230829/3_labels")

In [ ]:
cum_res.load_pypesto_problems(force_compile=True)

We first investigate some general information about the Results:

In [ ]:
non_existing = []
for index in range(cum_res.n_results):
    if not os.path.isfile(
            f"{res_dict}/{cum_res.model_name}_{index}_{cum_res.n_starts}starts.hdf5"
    ):
        non_existing.append(index)
cum_res.non_existing = non_existing

In [ ]:
cum_res.overview_failed_starts()

In [ ]:
# We want to compare how the optimization of the different models went. This is generally helpfull, as we expect a certain spread in the optimization, i.e. we should be suspicious if all models are fit perfectly or identical.
fvals = [result.optimize_result.fval[0] for result in cum_res.results]

ax = sns.histplot(fvals, kde=True)
plt.xlabel("Objective function value")
plt.ylabel("Relative frequency")
plt.title("Distribution of best objective function values")
ax.set_xticks([])
ax.set_yticks([])
plt.savefig("Figure3/fval_dist.pdf")

# compare to standard normal distribution
ks_result = stats.normaltest(fvals)
print(ks_result)

Here create the first part of Figure 4B: The cells ranked by objective function value.

It is similar to the visualization above, but shows clearer, which is the best median and worst fit. We want to have a deeper investigation to these three models to get a representative overview of the results.

In [ ]:
fvals = np.array(fvals)
fvals_sorted = np.sort(cum_res.fvals_best)
fvals_sorted_or = copy.deepcopy(fvals_sorted)
indices2plot = np.arange(len(fvals_sorted))
sorted_indices = np.round(np.argsort(cum_res.fvals_best)).astype(int)
figuresize = (5,1)
plot_points = True
n_points = 3
color_points = [
    "#007f5f", "#aacc00", "#fca311", "#17c3b2"
]
figure = plt.figure(figsize=figuresize)
indices = np.round(
    np.linspace(0, len(fvals_sorted) - 1, n_points).astype(int)
)
fvals_sorted = np.concatenate((fvals_sorted[0:6:2], fvals_sorted[12:-12:24], fvals_sorted[-6::2]))
indices2plot = np.concatenate((indices2plot[0:6:2], indices2plot[12:-12:24], indices2plot[-6::2]))
# plot the objective values as points
plt.plot(indices2plot, fvals_sorted, 'o', linewidth=2, markersize = 3, color = "#3B75AF")
plt.xlabel('Cell rank')
plt.ylabel('Objective value')

if plot_points:
    for i in range(n_points):
        plt.plot(
            indices[i], fvals_sorted_or[indices[i]], 'o',
            color=color_points[i], label=f"Cell {indices[i]}", markersize = 9
        )
plt.savefig("Model_230830/rank_cells.pdf")

Accordingly we want for each of the three fits see what the actual fit looks like. For this, we picked out two lipid classes DAG and LPE to see their exact fit. This corresponds to the middle building block of **Figure 4B**.

You can visualize other lipid classes by defining them through (e.g. adding PS as visualization)
1. Their base name (e.g. "PS")
2. The number of fatty acids they will have (e.g. 2)

In [ ]:
plt.rcParams["lines.linewidth"] = 3
for cell_index in [358, 214, 532]:
    for obs_name, obs_level in zip(["DAG", "LPE"], [2, 1]):  # Here you can place your lipid class of interest
        ax = cum_res.plot_time_trajectories_fit(
            cell_index = cell_index,
            obs_name = obs_name,
            obs_labels = obs_level
        )
        y = ax.get_ylim()
        ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax = y[1]))
        ax.get_legend().remove()
        ax.set_title(f"Cell {cell_index}, {obs_name}")
        ax.grid(False)
        ax.figure.set_size_inches(5, 5)
        ax.set_yticks([0 * y[1], 0.2 * y[1], 0.4 * y[1], 0.6 * y[1], 0.8 * y[1]])
        plt.tick_params(left = False, bottom = False, which='both')
        # make layout tight
        plt.tight_layout()
        plt.savefig(f"Model_230830/10_25_fit_cell_{cell_index}_{obs_name}.pdf")

The following block does the same thing, but generates the legend used in **Figure 4B** for us.

In [ ]:
plt.rcParams["lines.linewidth"] = 2
plt.rcParams["lines.markersize"] = 15
plt.rcParams["lines.marker"] = ""
ax = cum_res.plot_time_trajectories_fit(
    cell_index = 358,
    obs_name = "LPE",
    obs_labels = 1
)
# remove legend
ax.get_legend().remove()
# add legend manually
label_colors = [
    "black",
    "black",
    "black",
    (0.502, 0.733, 0.509),
    (0.944, 0.730, 0.418),
    (0.944, 0.558, 0.744)
]
labels = [
    "Measurement",
    "Fitted",
    "True",
    "Label 1",
    "Label 2",
    "Label 3"
]
linestyles = [
    "None",
    "-",
    "--",
    "-",
    "-",
    "-"
]
markers = ["o", "", "", "", "", ""]
handles = [
    mlines.Line2D([], [], color=color, linestyle=linestyle, label=label, marker=marker, markersize=8)
    for color, linestyle, label, marker in zip(label_colors, linestyles, labels, markers)
]
# remove grid
ax.grid(False)

ax.legend(handles=handles, loc='center left',
          bbox_to_anchor=(1, 0.5))
# set the legend box color to black
ax.legend_.get_frame().set_edgecolor('black')
# tight layout
plt.tight_layout()
# save figure
plt.savefig("Model_230830/fit_cell_358_LPE_legend.pdf")

Since it will not be possible to give a general overview of all fits of all lipid classes, we also want a scatter plot of all measurements. This will be the third square in the middle building block of **Figure 4B**

In [ ]:
def scatter_plot_single_cell_fit(
    cum_result,
    cell_index
):
    """
    Plot the fit of a single cell as a scatter plot.

    :param cum_result: The cumulative result object
    :param cell_index: The index of the cell to plot.
    :return: A matplotlib axis object
    """
    # get the result object
    result = cum_result.results[cell_index]
    # get the petab problem
    petab_problem = cum_result.petab_problems[cell_index]
    # get the objective
    objective = cum_result.pypesto_problem.objective
    # call the objective function with estimated parameters
    rdatas_est = objective(
        result.optimize_result[0].x,
        return_dict=True,
    )
    # call the objective function with true parameters
    rdatas_true = objective(
        petab_problem.x_nominal_free_scaled,
        return_dict=True,
    )

    return (rdatas_est, rdatas_true)

In [ ]:
color_cells = ["#007f5f", "#aacc00", "#fca311", "#17c3b2"]
for i_cell, cell_index in enumerate([358, 214, 532]):
    if i_cell < 2:
        continue
    rdatas_est, rdatas_true = scatter_plot_single_cell_fit(
        cum_res,
        cell_index = cell_index
    )
    figure = plt.figure()
    plt.scatter(
        rdatas_true["rdatas"][0].y,
        rdatas_est["rdatas"][0].y,
        color = color_cells[i_cell],
    )

    plt.tick_params(left = False, bottom = False, which='both')
    # change to log scale
    plt.xscale("log")
    plt.yscale("log")
    plt.title('')
    plt.ylabel('')
    plt.xlabel('')
    plt.title(f"Cell {cell_index}")
    figure.set_size_inches(5, 5)
    figure.axes[0].grid(False)
    plt.savefig(f"Model_230830/scatter_cell_{cell_index}.pdf")
    plt.show()
    plt.clf()


Last but not least, we want to compae whether there is any significant difference in the optimization of a model. For this we visualize the 1000 final optimization values of each local optimization start in a **Waterfall Plot**. In red we will see the overall best found value and the more red points we have the more often this exact likelihood value was found. This represents the right part of **Figure 4B**

**However**, while all these together give you a good overview of the model fit and whether the optimization process is able to recover the dynamics reliably, it does not tell us anything about the identifiability of the model, i.e. whether the parameters have been recovered equally well. For this we have created **Figure 5**

In [ ]:
plt.rcParams["lines.linewidth"] = 2
plt.rcParams['axes.edgecolor']='black'
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Set1.colors)
for cell_index in [358, 214, 532]:
    ax = cum_res.plot_waterfall(cell_index = cell_index)
    print(
        cum_res.results[cell_index].optimize_result.fval[0]
    )
    ax.grid(False)
    ax.figure.set_size_inches(4.5, 3)
    plt.savefig(f"Model_230830/waterfall_cell_{cell_index}.pdf")

The true parameters have been saved in the PEtab models, which is why we loaded them in the beginning. We make two arrays now, the best found parameter and the true parameters. We want to compare them
1. As a scatterplot (**Figure 5B**)
2. In terms of distribution (**Figure 5C**)

In [ ]:
cell_indices = len(cum_res.petab_problems)

par_est = [
    result.optimize_result[0].x for result in cum_res.results
]
par_true = [
    petab_problem.x_nominal_free_scaled for petab_problem in cum_res.petab_problems
]

In [ ]:
# turn them into arrays
par_est = np.array(par_est)
par_true = np.array(par_true)
# print their shapes
print(par_est.shape)
print(par_true.shape)

We first run the scatter plot. We turn it down to the first 5 parameters (line 1-3 in the following block) but one can easily extend this. For a given index i (line 5 in the second block), we plot this parameter as a scatterplot. We compare the optimal line (black) against the actual values (x-axis = true, y-axis = estimated values). This should reveal any bias we might encounter. As we can see, some parameters fit very well, while other are completely randomized. Since these are all the best found parameter values and their fit is still good, this points towards a practical unidentifiability of this parameter.

The following two block with i=0,1,2,3 reproduce **Figure 5B**

In [ ]:
cum_res.pypesto_problem.x_names[0:5]
par_est_few = par_est[:, 0:5]
par_true_few = par_true[:, 0:5]

cum_res.pypesto_problem.x_names[1]

In [ ]:
plt.rcParams["lines.linewidth"] = 2
plt.rcParams['axes.edgecolor']='black'
plt.rcParams["lines.markersize"] = 5
# plot the scatter plot colored by second dimension
i=1
fig, ax = plt.subplots(figsize=(5,5))
plt.scatter(
    par_true_few[:, i],
    par_est_few[:, i],
    label = f"{cum_res.pypesto_problem.x_names[i]}",
    alpha = 1,
    color = "#3B75AF",
)
# get the minimum and maximum value of the true parameters
min_true, min_est = np.min(par_true_few[:, i]), np.min(par_est_few[:, i])
max_true, max_est = np.max(par_true_few[:, i]), np.max(par_est_few[:, i])
min_val = min(min_true, min_est)
max_val = max(max_true, max_est)
# add a line that goes through (min_val, min_val) and (max_val, max_val)
plt.plot([min_val, max_val], [min_val, max_val], color="black")
# set x axis limits to -2 and -1
plt.xlim(min_val, max_val)
plt.xlim(min_true, max_true)  # only for unidentifiable
plt.ylim(min_val, max_val)
# remove grid
plt.grid(False)
# # remove tick labels
# ax.set_xticks([])
# ax.set_yticks([])
# add title
plt.title(cum_res.pypesto_problem.x_names[i])
# # add a legend
# plt.legend(loc='center left',
#           bbox_to_anchor=(1, 0.5))
plt.tick_params(left = False, bottom = False, which='both')
# tight layout
plt.tight_layout()
plt.savefig(f"Model_230830/scatter_plot_{cum_res.pypesto_problem.x_names[i]}.pdf")
plt.show()
plt.clf()

# calculate R^2
r_2 = np.corrcoef(par_true_few[:, i], par_est_few[:, i]) ** 2
print(f"Correlation Coefficient: {r_2}")

The previous section compared the parameters in a 1:1 setting. We might also be interested how well the general distribution of the parameters compare. For this we create **Figure 5C**

In [ ]:
plt.rcParams['axes.edgecolor']='black'
plt.rcParams["lines.linewidth"] = 10
matplotlib.rc('xtick', labelsize=10)
matplotlib.rc('ytick', labelsize=10)
ax = cum_res.box_plot_parameters()
ax.patch.set_edgecolor('black')
plt.xticks(rotation = 45)
plt.tick_params(left = False, bottom = False, which='both')
plt.grid(False)
plt.savefig(f"Model_230830/boxplots_half_wo_legend.pdf")